In [1]:
import sqlite3
from utils import normalize_label
import logging

logger = logging.getLogger(__name__)


conn = sqlite3.connect("data/alkfred.sqlite")


conn.row_factory = sqlite3.Row
cur = conn.cursor()

label = normalize_label("Lung Non-small Cell Carcinoma")


cur.execute("""
  SELECT t.ncit_id, t.label_display AS therapy, COUNT(DISTINCT f.eid) AS evidence_items
  FROM fact_evidence f
  JOIN dim_disease d ON d.doid = f.doid
  JOIN dim_evidence e ON e.eid = f.eid
  JOIN dim_therapy  t ON t.therapy_id = f.therapy_id
  WHERE d.label_disease_norm = ? AND e.evidence_level = ?
  GROUP BY t.ncit_id, t.label_display
  ORDER BY evidence_items DESC, therapy
  LIMIT 25
""", (label, "C"))
count = 0
for r in cur.fetchall():
    print(dict(r))
    count +=1
logger.info("Row count %s:", count)


2025-10-20 18:06:27,951 [INFO] Row count 6:


{'ncit_id': 'C74061', 'therapy': 'crizotinib', 'evidence_items': 14}
{'ncit_id': 'C101790', 'therapy': 'alectinib', 'evidence_items': 5}
{'ncit_id': 'C115112', 'therapy': 'ceritinib', 'evidence_items': 4}
{'ncit_id': 'C113655', 'therapy': 'lorlatinib', 'evidence_items': 2}
{'ncit_id': 'C71467', 'therapy': 'luminespib', 'evidence_items': 1}
{'ncit_id': 'C48401', 'therapy': 'retaspimycin hydrochloride', 'evidence_items': 1}


In [2]:
import json

with open("data/ALK_gene.json","r") as f:
    data = json.load(f)
    
def fast_parse(data: dict) -> dict:
    
       
    # iri = data.get("iri", "")
    # language = data.get("lang", "")
    for rec in data:
        if rec == "annotation":
            hgnc = data[rec]
        if rec == "iri":
            iri = data[rec]
        if rec == "lang":
            language = data[rec]
    
    results = { "iri" : iri, 
                "language": language,
                "hgnc": hgnc.get("HGNC_ID",[])
                
            }     
    return results

if __name__ == "__main__":
    print(fast_parse(data))
    
    
    


{'iri': 'http://purl.obolibrary.org/obo/NCIT_C38184', 'language': 'en', 'hgnc': ['HGNC:427']}


In [5]:
import sqlite3

from alkfred import config
from utils import normalize_label


conn = config.get_conn(config.default_db_path()) 
conn.row_factory = sqlite3.Row 
cur = conn.cursor() 

label = normalize_label("Lung Non-small Cell Carcinoma")

# levels = ("A","B","C","D")
# effects = ("SENSITIVITYRESPONSE", "RESISTANCE")

# sql = f"""
#         SELECT DISTINCT
#         f.doid,
#         f.eid,
#         f.variant_id,
#         v.label_gene_variant_norm,
#         e.significance
#         FROM fact_evidence f
#         JOIN dim_disease      d ON d.doid = f.doid
#         JOIN dim_evidence     e ON e.eid  = f.eid
#         JOIN dim_gene_variant v ON v.variant_id = f.variant_id
#         WHERE d.label_disease_norm = ?
#         AND e.evidence_level IN ({",".join("?"*len(levels))})
#         AND e.direction = 'SUPPORTS'
#         AND e.significance IN ({",".join("?"*len(effects))})
#         ORDER BY f.doid, f.eid, f.variant_id
#         LIMIT 200;
#         """

sql = """SELECT DISTINCT e.direction, e.significance, t.label_display
FROM fact_evidence f
JOIN dim_evidence e ON f.eid = e.eid
JOIN dim_gene_variant v ON f.variant_id = v.variant_id
JOIN dim_therapy t ON f.therapy_id = t.therapy_id
WHERE v.label_gene_variant_norm = 'alk_g1202r';
"""


# params = (label,)
# cur.execute(sql, params)
cur.execute(sql)
rows = [dict(r) for r in cur.fetchall()]
for r in rows:
    print(r)

{'direction': 'SUPPORTS', 'significance': 'RESISTANCE', 'label_display': 'crizotinib'}
{'direction': 'SUPPORTS', 'significance': 'RESISTANCE', 'label_display': 'ceritinib'}
{'direction': 'SUPPORTS', 'significance': 'RESISTANCE', 'label_display': 'alectinib'}
{'direction': 'SUPPORTS', 'significance': 'RESISTANCE', 'label_display': 'brigatinib'}
{'direction': 'SUPPORTS', 'significance': 'SENSITIVITY', 'label_display': 'tanespimycin'}
{'direction': 'SUPPORTS', 'significance': 'RESISTANCE', 'label_display': 'lorlatinib'}


In [8]:
import sqlite3

from alkfred import config
from utils import normalize_label


conn = config.get_conn(config.default_db_path()) 
conn.row_factory = sqlite3.Row 
cur = conn.cursor() 

label = normalize_label("Lung Non-small Cell Carcinoma")

# levels = ("A","B","C","D")
# effects = ("SENSITIVITYRESPONSE", "RESISTANCE")

# sql = f"""
#         SELECT DISTINCT
#         f.doid,
#         f.eid,
#         f.variant_id,
#         v.label_gene_variant_norm,
#         e.significance
#         FROM fact_evidence f
#         JOIN dim_disease      d ON d.doid = f.doid
#         JOIN dim_evidence     e ON e.eid  = f.eid
#         JOIN dim_gene_variant v ON v.variant_id = f.variant_id
#         WHERE d.label_disease_norm = ?
#         AND e.evidence_level IN ({",".join("?"*len(levels))})
#         AND e.direction = 'SUPPORTS'
#         AND e.significance IN ({",".join("?"*len(effects))})
#         ORDER BY f.doid, f.eid, f.variant_id
#         LIMIT 200;
#         """

sql = """WITH base AS (
  SELECT
    f.eid,
    f.variant_id,
    v.label_gene_variant_norm AS variant,
    t.label_display           AS therapy,
    e.significance,
    e.description
  FROM fact_evidence f
  JOIN dim_evidence     e ON e.eid        = f.eid
  JOIN dim_gene_variant v ON v.variant_id = f.variant_id
  JOIN dim_therapy      t ON t.therapy_id = f.therapy_id
  JOIN dim_disease      d ON d.doid       = f.doid
  WHERE d.label_disease_norm = ?                          -- e.g., normalize_label("Lung Non-small Cell Carcinoma")
    AND e.direction = 'SUPPORTS'
    AND e.significance IN ('RESISTANCE','SENSITIVITY','SENSITIVITYRESPONSE')
)
SELECT
  variant,
  -- distinct EIDs by effect (so one evidence doesn’t double count across therapies)
  COUNT(DISTINCT CASE WHEN significance='RESISTANCE' THEN eid END) AS resistant_eids,
  COUNT(DISTINCT CASE WHEN significance IN ('SENSITIVITY','SENSITIVITYRESPONSE') THEN eid END) AS sensitive_eids,
  -- list all therapies per effect (deduped)
  COALESCE(GROUP_CONCAT(DISTINCT CASE WHEN significance='RESISTANCE' THEN therapy END), '') AS resistant_therapies,
  COALESCE(GROUP_CONCAT(DISTINCT CASE WHEN significance IN ('SENSITIVITY','SENSITIVITYRESPONSE') THEN therapy END), '') AS sensitive_therapies
FROM base
GROUP BY variant
ORDER BY resistant_eids DESC, sensitive_eids DESC, variant
LIMIT 500;"""


params = (label,)
# cur.execute(sql, params)
cur.execute(sql, params)
rows = [dict(r) for r in cur.fetchall()]
for r in rows:
    print(r)

{'variant': 'eml4_alk_fusion', 'resistant_eids': 20, 'sensitive_eids': 15, 'resistant_therapies': 'crizotinib,ceritinib,luminespib,lorlatinib,alectinib,alk inhibitor tae684', 'sensitive_therapies': 'alectinib,lorlatinib,crizotinib,whi-p154,retaspimycin hydrochloride,alvespimycin,ceritinib,alk inhibitor tae684,brigatinib,tanespimycin'}
{'variant': 'alk_i1171', 'resistant_eids': 3, 'sensitive_eids': 4, 'resistant_therapies': 'alectinib,crizotinib', 'sensitive_therapies': 'ceritinib,alk inhibitor tae684'}
{'variant': 'alk_c1156y', 'resistant_eids': 3, 'sensitive_eids': 2, 'resistant_therapies': 'ceritinib,luminespib,crizotinib,lorlatinib', 'sensitive_therapies': 'lorlatinib,crizotinib'}
{'variant': 'alk_t1151dup', 'resistant_eids': 3, 'sensitive_eids': 1, 'resistant_therapies': 'crizotinib,alectinib,alk inhibitor tae684', 'sensitive_therapies': 'tanespimycin'}
{'variant': 'alk_amplification', 'resistant_eids': 3, 'sensitive_eids': 0, 'resistant_therapies': 'crizotinib', 'sensitive_therapi

In [9]:
import sqlite3

from alkfred import config
from utils import normalize_label


conn = config.get_conn(config.default_db_path()) 
conn.row_factory = sqlite3.Row 
cur = conn.cursor() 

label = normalize_label("Lung Non-small Cell Carcinoma")

# levels = ("A","B","C","D")
# effects = ("SENSITIVITYRESPONSE", "RESISTANCE")

# sql = f"""
#         SELECT DISTINCT
#         f.doid,
#         f.eid,
#         f.variant_id,
#         v.label_gene_variant_norm,
#         e.significance
#         FROM fact_evidence f
#         JOIN dim_disease      d ON d.doid = f.doid
#         JOIN dim_evidence     e ON e.eid  = f.eid
#         JOIN dim_gene_variant v ON v.variant_id = f.variant_id
#         WHERE d.label_disease_norm = ?
#         AND e.evidence_level IN ({",".join("?"*len(levels))})
#         AND e.direction = 'SUPPORTS'
#         AND e.significance IN ({",".join("?"*len(effects))})
#         ORDER BY f.doid, f.eid, f.variant_id
#         LIMIT 200;
#         """

sql = """SELECT d.doid, d.label_display AS disease, t.label_display AS therapy,
       e.direction, e.significance, e.evidence_level, f.eid as fact_eid
FROM fact_evidence f
JOIN dim_disease  d ON d.doid = f.doid
JOIN dim_evidence e ON e.eid  = f.eid
JOIN dim_therapy  t ON t.therapy_id = f.therapy_id
JOIN dim_gene_variant v ON v.variant_id = f.variant_id
WHERE v.label_gene_variant_norm = 'alk_g1202r'
  AND d.label_disease_norm = 'lung_non-small_cell_carcinoma'  -- your normalized label
ORDER BY therapy, fact_eid;"""


# params = (label,)
# cur.execute(sql, params)
cur.execute(sql)
rows = [dict(r) for r in cur.fetchall()]
for r in rows:
    print(r)